In [1]:
import os

class CFG_good:
    num_workers=10
    path="labse_v4_good"
    config_path=os.path.join(path, "config.pth")
    model="sentence-transformers/LaBSE"
    batch_size=32
    fc_dropout=0.1
    max_len=66
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

class CFG_brand:
    num_workers=10
    path="labse_v4_brand"
    config_path=os.path.join(path, "config.pth")
    model="sentence-transformers/LaBSE"
    batch_size=32
    fc_dropout=0.1
    max_len=66
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

In [2]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.13.3
transformers.__version__: 4.30.1
env: TOKENIZERS_PARALLELISM=true


In [3]:
def probs2str_good(results, texts): # (4501, 70, 3)
    predictions = []
    classes = np.argmax(results, axis=-1)
    input_ids_list = [CFG_good.tokenizer(text, 
                           add_special_tokens=True,
                           max_length=CFG_good.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)["input_ids"] for text in texts]

    for clas, input_ids in zip(classes, input_ids_list):
        entities = convert_ids_to_entities(clas, input_ids)

        entities = list(itertools.chain(*entities))
        prediction = CFG_good.tokenizer.decode(entities, skip_special_tokens=True)
        prediction = re.sub(" - ", "-", prediction)
        prediction = re.sub(" & ", "&", prediction)
        prediction = re.sub("##", "", prediction)
        predictions.append(prediction)
    
    return predictions

def probs2str_brand(results, texts): # (4501, 70, 3)
    predictions = []
    classes = np.argmax(results, axis=-1)
    input_ids_list = [CFG_brand.tokenizer(text, 
                           add_special_tokens=True,
                           max_length=CFG_brand.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)["input_ids"] for text in texts]

    for clas, input_ids in zip(classes, input_ids_list):
        entities = convert_ids_to_entities(clas, input_ids)

        entities = list(itertools.chain(*entities))
        prediction = CFG_brand.tokenizer.decode(entities, skip_special_tokens=True)
        prediction = re.sub(" - ", "-", prediction)
        prediction = re.sub(" & ", "&", prediction)
        prediction = re.sub("##", "", prediction)
        predictions.append(prediction)
    
    return predictions

def convert_ids_to_entities(ids, tokens=None, o_id=0, b_id=1, i_id=2):
    entities = []
    entity = []
    is_entity_started = False
    for index, token in enumerate(ids):
        if not is_entity_started and token == b_id:
            is_entity_started = True
            entity.append(index)
        elif is_entity_started and token == i_id:
            entity.append(index)
    
        if (is_entity_started and token == o_id) or (index == (len(ids) - 1)):
            is_entity_started = False
            entities.append(entity)
            entity = []
                
    if tokens is not None:
        tokens = np.array(tokens)
        token_entities = []
        for entity in entities:
            entity = np.array(entity)
            if len(entity) > 0:
                token_entity = tokens[entity]
                token_entities.append(token_entity.tolist())
            
        return token_entities
            
    return entities

In [4]:
CFG_good.tokenizer = AutoTokenizer.from_pretrained(os.path.join(CFG_good.path, "tokenizer"))
CFG_brand.tokenizer = AutoTokenizer.from_pretrained(os.path.join(CFG_brand.path, "tokenizer"))

In [5]:
def get_logger(filename=os.path.join(CFG_good.path, "inference")):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [6]:
test = pd.read_csv('test_dataset.csv')
test.name = test.name.apply(lambda x: x.lower())
submission = pd.read_csv('sample_submission.csv')

unsup = pd.read_csv("train_unsupervised_dataset.csv")
unsup.name = unsup.name.apply(lambda x: x.lower())

print(f"test.shape: {test.shape}")
display(test.head())

test.shape: (5000, 2)


,id,name
0,0,"469-210 ермак клей универсальный, 15мл, блистер"
1,1,торт сладушка зимняя вишня 700г
2,2,"смеситель ""calorie"" 1023 а06 д/кухни"
3,3,лимон 50гр бар
4,4,"коньяк сараджишвили 5 лет 0,5л грузия"


In [7]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text, 
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False,
                          truncation=True)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.name = df['name'].values

    def __len__(self):
        return len(self.name)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.name[item])
        return inputs

In [8]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 3)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [9]:
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.softmax(-1).to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [10]:
test_dataset_good = TestDataset(CFG_good, test)
test_loader_good = DataLoader(test_dataset_good,
                         batch_size=CFG_good.batch_size,
                         shuffle=False,
                         num_workers=CFG_good.num_workers, pin_memory=True, drop_last=False)

test_dataset_brand = TestDataset(CFG_brand, test)
test_loader_brand = DataLoader(test_dataset_brand,
                         batch_size=CFG_brand.batch_size,
                         shuffle=False,
                         num_workers=CFG_brand.num_workers, pin_memory=True, drop_last=False)

In [10]:
unsup_dataset_good = TestDataset(CFG_good, unsup)
unsup_loader_good = DataLoader(unsup_dataset_good,
                         batch_size=CFG_good.batch_size,
                         shuffle=False,
                         num_workers=CFG_good.num_workers, pin_memory=True, drop_last=False)

unsup_dataset_brand = TestDataset(CFG_brand, unsup)
unsup_loader_brand = DataLoader(unsup_dataset_brand,
                         batch_size=CFG_brand.batch_size,
                         shuffle=False,
                         num_workers=CFG_brand.num_workers, pin_memory=True, drop_last=False)

In [11]:
predictions_good = []
for fold in CFG_good.trn_fold:
    model = CustomModel(CFG_good, config_path=CFG_good.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_good.path, f"{CFG_good.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader_good, model, device)
    predictions_good.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_good = np.mean(predictions_good, axis=0)

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

In [11]:
predictions_good = []
for fold in CFG_good.trn_fold:
    model = CustomModel(CFG_good, config_path=CFG_good.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_good.path, f"{CFG_good.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(unsup_loader_good, model, device)
    predictions_good.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_good = np.mean(predictions_good, axis=0)

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

In [12]:
predictions_brand = []
for fold in CFG_brand.trn_fold:
    model = CustomModel(CFG_brand, config_path=CFG_brand.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_brand.path, f"{CFG_brand.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader_brand, model, device)
    predictions_brand.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_brand = np.mean(predictions_brand, axis=0)

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

In [12]:
predictions_brand = []
for fold in CFG_brand.trn_fold:
    model = CustomModel(CFG_brand, config_path=CFG_brand.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_brand.path, f"{CFG_brand.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(unsup_loader_brand, model, device)
    predictions_brand.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_brand = np.mean(predictions_brand, axis=0)

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

In [13]:
test_texts = test['name'].values

pred_goods = probs2str_good(predictions_good, test_texts)
pred_brands = probs2str_brand(predictions_brand, test_texts)

In [13]:
unsup_texts = unsup['name'].values

pred_goods = probs2str_good(predictions_good, unsup_texts)
pred_brands = probs2str_brand(predictions_brand, unsup_texts)

In [14]:
unsup["good"] = pred_goods
unsup["brand"] = pred_brands
unsup

,id,name,good,brand
0,0,зубная щетка орал би три эффект деликатное отб...,зубная щетка,орал би
1,1,салфетки vister влажные для ко,салфетки,vister
2,2,платье женское dr8517k 7л8999 светло-серый 449...,платье,
3,3,"лакомство ""деревенские лакомства"" д/собак мини...",лакомство,деревенские лакомства
4,4,суппорт гитарный ergo play troster,суппорт,ergo
...,...,...,...,...
999995,999995,f-2296 спонж д/макияжа фигурный (шт),спонж,
999996,999996,4 5702737510597 69.88 дифф/arom/1601,,arom
999997,999997,матрас надувной 540*74см tropical bird запл.д/...,матрас,tropical bird
999998,999998,"пододеяльник стм страйп 3-сп, размер: 796х185с...",пододеяльник,


In [15]:
unsup.to_csv("unsup_pred_rubert.csv", index=False)

In [14]:
submission["good"] = pred_goods
submission["brand"] = pred_brands
submission

,id,good,brand
0,0,клей,ермак
1,1,торт,сладушка
2,2,смеситель,calorie
3,3,лимон,
4,4,коньяк,сараджишвили
...,...,...,...
4995,4995,рамка,
4996,4996,напиток,red bull
4997,4997,наконечники,
4998,4998,шоколад,риттерспорт


In [15]:
submission.to_csv("NER_baseline_labse_v4.csv", index=False)